In [105]:
print("Hello World")

Hello World


In [106]:
readFile="Data/parquet/subSampleWholeDataSet"

SKlearn has a larger variety of Ensemble tree methods than Pyspark. In Pyspark Gradient boosted Trees gave excellent results but the run time was far too long even on our subset. I have refactored the code in SKlearn and will try the larger range of Tree methods, with 20 dimensions. I'll use the parameters from this document.
https://scikit-learn.org/stable/modules/ensemble.html#
We need to set the hashseed so the mode can be saved and reloaded.
On this sheet we will run a basic CV for tuning and compare the performance of Decision Trees
https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html
We then have 2 averaging trees,
Random Forest which we used in Pyspark, and Extra Trees which introduduces a an extra element of randomness.
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn.ensemble.RandomForestClassifier
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.ExtraTreesClassifier.html#sklearn.ensemble.ExtraTreesClassifier
I will run the boosing trees after



We need to set the hashseed so the mode can be saved and reloaded.

In [107]:
! PYTHONHASHSEED=0
import pandas as pd
import numpy as np

In [108]:
dataset=pd.read_parquet(readFile, engine='pyarrow')

In [109]:
 from sklearn.preprocessing import LabelEncoder, OneHotEncoder

We will use the slightly extended number of features

In [110]:
categoricalCols = [
    "wlan_tag_length",
    "wlan_country_info_fnm",
    "wlan_da",
    "wlan_sa",
    "wlan_bssid",
    "ip_src",
    "ip_dst",
    "http_host",
    "http_request_line",
    "http_request_method",
    "http_request_version",
    "ssdp",
    "http_request_full_uri"
]


In [111]:
numericCols=[
    "wlan_fixed_reason_code",
    "radiotap_channel_freq",
    "udp_length",
    "ip_ttl",
    "udp_dstport",
    "udp_time_delta"
]

In [112]:
y=dataset['label']
x=dataset[categoricalCols+numericCols]
x[categoricalCols].astype("category")

,wlan_tag_length,wlan_country_info_fnm,wlan_da,wlan_sa,wlan_bssid,ip_src,ip_dst,http_host,http_request_line,http_request_method,http_request_version,ssdp,http_request_full_uri
0,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
1,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
3,NA,NA,50:3e:aa:e4:01:93,00:0c:29:cf:08:aa,0c:9d:92:54:fe:34,192.168.2.130,192.168.2.73,NA,NA,NA,NA,NA,NA
4,NA,NA,50:3e:aa:e4:01:93,00:0c:29:cf:08:aa,0c:9d:92:54:fe:34,192.168.2.130,192.168.2.73,NA,NA,NA,NA,NA,NA
...,...,...,...,...,...,...,...,...,...,...,...,...,...
632402,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
632403,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
632404,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
632405,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [113]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
categorical_transformer = OneHotEncoder(handle_unknown="ignore")
from sklearn.preprocessing import FunctionTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ("cat", categorical_transformer, categoricalCols)
    ]
)

We have our pieline set up and now make our test train split.

In [114]:
from sklearn.model_selection import train_test_split, GridSearchCV
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)


In [115]:
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV

In [63]:

DT = Pipeline(steps=[("preprocess", preprocessor), ("DTclassifier", DecisionTreeClassifier(random_state=0))])

DT have few parameters so we will try gini and entropy (log loss produced errors

In [65]:
param_grid = { 
    'DTclassifier__criterion' :['gini','entropy'],
    'DTclassifier__max_features' :['auto', 'sqrt', 'log2']
}

In SKlearn we have a lot more scoring metrics, 
https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
I am using recall rather than the area under PR curve as catching the attack packets is my main concern. 

In [72]:
%%time
CV_DT = GridSearchCV(estimator=DT, param_grid=param_grid, cv= 5)
CV_DT.fit(X_train, y_train)

CPU times: user 1min 9s, sys: 3.55 s, total: 1min 12s
Wall time: 1min 12s


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocess',
                                        ColumnTransformer(transformers=[('cat',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['wlan_tag_length',
                                                                          'wlan_country_info_fnm',
                                                                          'wlan_da',
                                                                          'wlan_sa',
                                                                          'wlan_bssid',
                                                                          'ip_src',
                                                                          'ip_dst',
                                                                          'http_host',
                             

In [73]:
CV_DT.best_params_

{'DTclassifier__criterion': 'gini', 'DTclassifier__max_features': 'auto'}

In [74]:
TunedDT = Pipeline(steps=[("preprocess", preprocessor), ("DTclassifier", DecisionTreeClassifier(max_depth=None, min_samples_split=2,criterion='gini',max_features='auto',random_state=0))])

In [75]:
%%time
TunedDT.fit(X_train, y_train)
print("The model score for a DecisionTreeClassifier: %.3f" % TunedDT.score(X_test, y_test))
testPredictions=TunedDT.predict(X_test)

The model score for a DecisionTreeClassifier: 0.991
CPU times: user 3.04 s, sys: 176 ms, total: 3.21 s
Wall time: 3.21 s


In [76]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import matthews_corrcoef
tn, fp, fn, tp = confusion_matrix(y_test, testPredictions).ravel()
precision,recall,fscore,support = precision_recall_fscore_support(y_test, testPredictions, average='binary')
MCC = matthews_corrcoef(y_test, testPredictions)
print("There are "+str(tn)+ " true negatives")
print("There are "+str(tp)+ " true positives")
print("There are "+str(fn)+ " false negatives")
print("There are "+str(fp)+ " false positives")
print("The precision is "+str(precision))
print("The recall is "+str(recall))
print("The fscore is "+str(fscore))
print("The Matthews correlation coefficient is "+str(MCC))

There are 119065 true negatives
There are 6298 true positives
There are 803 false negatives
There are 316 false positives
The precision is 0.9522225582098579
The recall is 0.8869173355865371
The fscore is 0.9184104994531536
The Matthews correlation coefficient is 0.9143841723115039


So the DecisionTreeClassifier gives as good results as the other trees, we probably wont use it as there are fewer parameters to tune for the final model but it gives a good baseline. Next Random Forest

In [22]:

RF = Pipeline(steps=[("preprocess", preprocessor), ("RFclassifier", RandomForestClassifier(random_state=0))])

I just look at the max features here, the n_estimators would be good to test, but it is timecomsuming running multiple iterations over the default value of 100. My reading suggests the higher the value the better, so I will tune this on the final model.
https://www.analyticsvidhya.com/blog/2015/06/tuning-random-forest-model/#:~:text=n_estimators%20%3A,but%20makes%20your%20code%20slower.

In [89]:
param_grid = { 
    'RFclassifier__max_features' :['auto', 'sqrt', 'log2']
}


In [90]:
%%time
CV_RF = GridSearchCV(estimator=RF, param_grid=param_grid, cv= 5,n_jobs=-1)
CV_RF.fit(X_train, y_train)

CPU times: user 2min 24s, sys: 2.02 s, total: 2min 26s
Wall time: 9min 56s


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocess',
                                        ColumnTransformer(transformers=[('cat',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['wlan_tag_length',
                                                                          'wlan_country_info_fnm',
                                                                          'wlan_da',
                                                                          'wlan_sa',
                                                                          'wlan_bssid',
                                                                          'ip_src',
                                                                          'ip_dst',
                                                                          'http_host',
                             

In [91]:
CV_RF.best_params_

{'RFclassifier__max_features': 'log2'}

In [92]:
TunedRF = Pipeline(steps=[("preprocess", preprocessor), ("RFclassifier", RandomForestClassifier(max_depth=None, min_samples_split=2,max_features='log2',criterion='gini',n_estimators=100,random_state=0))])

In [93]:
%%time
TunedRF.fit(X_train, y_train)
print("The model score for a RandomForestClassifier: %.3f" % TunedRF.score(X_test, y_test))
testPredictions=TunedRF.predict(X_test)

The model score for a RandomForestClassifier: 0.991
CPU times: user 2min 31s, sys: 59 ms, total: 2min 31s
Wall time: 2min 31s


In [94]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import matthews_corrcoef
tn, fp, fn, tp = confusion_matrix(y_test, testPredictions).ravel()
precision,recall,fscore,support = precision_recall_fscore_support(y_test, testPredictions, average='binary')
MCC = matthews_corrcoef(y_test, testPredictions)
print("There are "+str(tn)+ " true negatives")
print("There are "+str(tp)+ " true positives")
print("There are "+str(fn)+ " false negatives")
print("There are "+str(fp)+ " false positives")
print("The precision is "+str(precision))
print("The recall is "+str(recall))
print("The fscore is "+str(fscore))
print("The Matthews correlation coefficient is "+str(MCC))

There are 119075 true negatives
There are 6299 true positives
There are 802 false negatives
There are 306 false positives
The precision is 0.953671461014383
The recall is 0.8870581608224194
The fscore is 0.9191594921932
The Matthews correlation coefficient is 0.9152049719746969


We get a slightly better performance, but with more estimators we should be able to improve further
The Extra trees is similar for the random forest so we won't cross validate

In [97]:
TunedET = Pipeline(steps=[("preprocess", preprocessor), ("ETclassifier", ExtraTreesClassifier(max_depth=None, min_samples_split=2,max_features='log2',criterion='gini',n_estimators=100,random_state=0))])

In [98]:
%%time
TunedET.fit(X_train, y_train)
print("The model score for a ExtraTreesClassifier: %.3f" % TunedET.score(X_test, y_test))
testPredictions=TunedET.predict(X_test)

The model score for a ExtraTreesClassifier: 0.991
CPU times: user 2min 41s, sys: 56.2 ms, total: 2min 41s
Wall time: 2min 41s


In [99]:
tn, fp, fn, tp = confusion_matrix(y_test, testPredictions).ravel()
precision,recall,fscore,support = precision_recall_fscore_support(y_test, testPredictions, average='binary')
MCC = matthews_corrcoef(y_test, testPredictions)
print("There are "+str(tn)+ " true negatives")
print("There are "+str(tp)+ " true positives")
print("There are "+str(fn)+ " false negatives")
print("There are "+str(fp)+ " false positives")
print("The precision is "+str(precision))
print("The recall is "+str(recall))
print("The fscore is "+str(fscore))
print("The Matthews correlation coefficient is "+str(MCC))

There are 119075 true negatives
There are 6296 true positives
There are 805 false negatives
There are 306 false positives
The precision is 0.9536504089669797
The recall is 0.8866356851147725
The fscore is 0.9189228636065094
The Matthews correlation coefficient is 0.9149641942105532


so the results are almost the same, looks like three scores TP to FN. Next we move onto the boosted Trees. The GBT gave me my best results in pyspark, but with a massive run time. 

We then have two boosting methods Adaboost and Gradient boosted trees which produced excellent results in a very long time in Pyspark
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html#sklearn.ensemble.AdaBoostClassifier
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html#sklearn.ensemble.GradientBoostingClassifier
there is another that sounds good for my data, gradient boosted trees for larger datasets, but it did not work on my data as it is fairly new implementationand does not work on a sparse matrix.
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.HistGradientBoostingClassifier.html#sklearn.ensemble.HistGradientBoostingClassifier
First we look at adaboost. I will set the n_estimators to 100, so it is the same as the RF examples and we will adjust the learning rate and the Max Features in the CV

In [150]:

AB = Pipeline(steps=[("preprocess", preprocessor), ("ABclassifier", AdaBoostClassifier(random_state=0))])

So with the Adaboost we will look at the learning rate and max features, the documentation says there is a trade of with learning rate and number of estimators. 

In [151]:
param_grid = { 
    'ABclassifier__learning_rate' :[0.001,0.1,1]
}


In [152]:
%%time
CV_AB = GridSearchCV(estimator=AB, param_grid=param_grid, cv= 5,n_jobs=-1)
CV_AB.fit(X_train, y_train)

CPU times: user 27.9 s, sys: 2.82 s, total: 30.7 s
Wall time: 2min 53s


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocess',
                                        ColumnTransformer(transformers=[('cat',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['wlan_tag_length',
                                                                          'wlan_country_info_fnm',
                                                                          'wlan_da',
                                                                          'wlan_sa',
                                                                          'wlan_bssid',
                                                                          'ip_src',
                                                                          'ip_dst',
                                                                          'http_host',
                             

In [153]:
CV_AB.best_params_

{'ABclassifier__learning_rate': 1}

In [157]:
TunedAB = Pipeline(steps=[("preprocess", preprocessor), ("ABclassifier", AdaBoostClassifier(n_estimators=100,learning_rate=1,random_state=0))])

In [158]:
%%time
TunedAB.fit(X_train, y_train)
print("The model score for a AdaBoostClassifier: %.3f" % TunedAB.score(X_test, y_test))
testPredictions=TunedAB.predict(X_test)

The model score for a AdaBoostClassifier: 0.986
CPU times: user 49.2 s, sys: 1.37 s, total: 50.6 s
Wall time: 50.6 s


In [159]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import matthews_corrcoef
tn, fp, fn, tp = confusion_matrix(y_test, testPredictions).ravel()
precision,recall,fscore,support = precision_recall_fscore_support(y_test, testPredictions, average='binary')
MCC = matthews_corrcoef(y_test, testPredictions)
print("There are "+str(tn)+ " true negatives")
print("There are "+str(tp)+ " true positives")
print("There are "+str(fn)+ " false negatives")
print("There are "+str(fp)+ " false positives")
print("The precision is "+str(precision))
print("The recall is "+str(recall))
print("The fscore is "+str(fscore))
print("The Matthews correlation coefficient is "+str(MCC))

There are 118694 true negatives
There are 6002 true positives
There are 1099 false negatives
There are 687 false positives
The precision is 0.8972940648826432
The recall is 0.8452330657653851
The fscore is 0.8704858593183465
The Matthews correlation coefficient is 0.8634566932345586


So Adaboost has given worse results,Next we can try the Gradient boosted Trees

In [164]:

GB = Pipeline(steps=[("preprocess", preprocessor), ("GBclassifier", GradientBoostingClassifier(random_state=0))])

So with the Gradient Boosts we will look at the learning rate and max depth, while AB default is 1, it is 0.1 for GB and for the RF algorthims the default max depth is None it is 3 for GB

In [165]:
param_grid = { 
    'GBclassifier__learning_rate' :[0.01,0.1,1],
    'GBclassifier__max_depth' :[3,9]
}


In [166]:
%%time
CV_GB = GridSearchCV(estimator=GB, param_grid=param_grid, cv= 5,n_jobs=6)
CV_GB.fit(X_train, y_train)

CPU times: user 3min 31s, sys: 2.93 s, total: 3min 34s
Wall time: 29min 38s


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocess',
                                        ColumnTransformer(transformers=[('cat',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['wlan_tag_length',
                                                                          'wlan_country_info_fnm',
                                                                          'wlan_da',
                                                                          'wlan_sa',
                                                                          'wlan_bssid',
                                                                          'ip_src',
                                                                          'ip_dst',
                                                                          'http_host',
                             

In [167]:
CV_GB.best_params_

{'GBclassifier__learning_rate': 0.1, 'GBclassifier__max_depth': 9}

In [168]:
TunedGB = Pipeline(steps=[("preprocess", preprocessor), ("GBclassifier", GradientBoostingClassifier(n_estimators=100,learning_rate=0.1,max_depth=9,random_state=0))])

In [169]:
%%time
TunedGB.fit(X_train, y_train)
print("The model score for a GradientBoostingClassifier: %.3f" % TunedGB.score(X_test, y_test))
testPredictions=TunedGB.predict(X_test)

The model score for a GradientBoostingClassifier: 0.991
CPU times: user 3min 21s, sys: 1.63 s, total: 3min 23s
Wall time: 3min 23s


In [170]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import matthews_corrcoef
tn, fp, fn, tp = confusion_matrix(y_test, testPredictions).ravel()
precision,recall,fscore,support = precision_recall_fscore_support(y_test, testPredictions, average='binary')
MCC = matthews_corrcoef(y_test, testPredictions)
print("There are "+str(tn)+ " true negatives")
print("There are "+str(tp)+ " true positives")
print("There are "+str(fn)+ " false negatives")
print("There are "+str(fp)+ " false positives")
print("The precision is "+str(precision))
print("The recall is "+str(recall))
print("The fscore is "+str(fscore))
print("The Matthews correlation coefficient is "+str(MCC))

There are 119083 true negatives
There are 6256 true positives
There are 845 false negatives
There are 298 false positives
The precision is 0.9545315837656393
The recall is 0.8810026756794818
The fscore is 0.9162943976565361
The Matthews correlation coefficient is 0.9123444003141361


so we have slightly worse results for the Gradient Boosted Trees, we know from pyspark it is possible to get better results than, this but the overall conclusion is that that the trees all give similar performance on my subsample. 